## Preliminary gradient calculations with near-real-time Wave Glider ADCP data

* load cleaned up ADCP files from

first cut for IOP1 by Tom, 10/20/2022  

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import cftime
import requests
import cartopy.crs as ccrs                   # import projections
import cartopy
import gsw
import functions  # requires functions.py from this directory

In [2]:
# %matplotlib inline
%matplotlib qt5
plt.rcParams['figure.figsize'] = (7,4)
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 400
plt.close('all')

__figdir__ = '../plots/vel_fits/' 
savefig_args = {'bbox_inches':'tight', 'pad_inches':0.2}
plotfiletype='png'

In [3]:
savefig = True
zoom = True
if zoom:
    xmin, xmax = (-127,-121)
    ymin, ymax = (36.25,38.5)
    levels = np.linspace(14,17,21)-2.5
else:
    xmin, xmax = (-127,-121)
    ymin, ymax = (35, 41)
    levels = np.linspace(13,18,11)

    

Payload 2 Table 1 has met, ctd variables  
Payload 2 Table 2 has RDI variables

In [4]:
# List of WGs
input_list = ['WHOI-ASL22','WHOI-ASL32','SV3-1043','STOKES', 'PLANCK', 'PASCAL', 'KELVIN', 'CARSON']
url_prefix = 'http://smode.whoi.edu:8080/thredds/fileServer/IOP1_2022/waveglider/'
tab1_postfix = '_PLD2_TAB1.nc'
prefix = 'adcp_'
position_postfix = '_position.nc'
WG_list = ['WHOI22','WHOI32','WHOI43','STOKES', 'PLANCK', 'PASCAL', 'CARSON'] #leave Kelvin out because of ADCP problem in IOP1
path='../data/raw/WG_NRT/'

In [5]:
# For some reason, reading the files over the internet directly is not working well
# Download instead

n=0
file_list2 = []
for WG in WG_list:
    file2 = path+prefix+WG+'.nc'
    file_list2.append(file2)


In [6]:
file_list2

['../data/raw/WG_NRT/adcp_WHOI22.nc',
 '../data/raw/WG_NRT/adcp_WHOI32.nc',
 '../data/raw/WG_NRT/adcp_WHOI43.nc',
 '../data/raw/WG_NRT/adcp_STOKES.nc',
 '../data/raw/WG_NRT/adcp_PLANCK.nc',
 '../data/raw/WG_NRT/adcp_PASCAL.nc',
 '../data/raw/WG_NRT/adcp_CARSON.nc']

In [7]:
# Read in cleaned ADCP files from all WG
n=0
for WG in WG_list:
    input_WG=input_list[n]
    #file1 = file_list1[n]
    file2 = file_list2[n]
    #file3 = file_list3[n]
    #varstr = 'met_'+WG
    #ds_met_temp=xr.open_dataset(file1,decode_times=True)
    #locals()[varstr]=fix_ds_time(ds_met_temp) #Drop nonunique values and sort time
    varstr = 'adcp_'+WG
    locals()[varstr]=xr.open_dataset(file2,decode_times=True) #Time and z already fixed in WG_realtime_cleanup.ipynb
    #varstr = 'pos_'+WG
    #ds_pos_temp=xr.open_dataset(file3,decode_times=True)
    #locals()[varstr]=fix_ds_time(ds_pos_temp) #Drop nonunique values and sort time
    n=n+1
    print(file2)

../data/raw/WG_NRT/adcp_WHOI22.nc
../data/raw/WG_NRT/adcp_WHOI32.nc
../data/raw/WG_NRT/adcp_WHOI43.nc
../data/raw/WG_NRT/adcp_STOKES.nc
../data/raw/WG_NRT/adcp_PLANCK.nc
../data/raw/WG_NRT/adcp_PASCAL.nc
../data/raw/WG_NRT/adcp_CARSON.nc


In [8]:
# Now we can access these in a loop using syntax like:
# eval('adcp_'+WG_list[7])

In [9]:
eval('adcp_'+WG_list[2])

<xarray.Dataset>
Dimensions:               (time: 1643, z: 50)
Coordinates:
  * time                  (time) datetime64[ns] 2022-09-22T03:46:08 ... 2022-...
Dimensions without coordinates: z
Data variables: (12/16)
    year                  (time) float32 ...
    day                   (time) float32 ...
    hour                  (time) float32 ...
    minute                (time) float32 ...
    sec                   (time) float32 ...
    RECORD                (time) float32 ...
    ...                    ...
    current_east          (z, time) float32 ...
    current_north         (z, time) float32 ...
    z_matrix              (z, time) float32 ...
    Longitude             (time) float64 ...
    Latitude              (time) float64 ...
    depth                 (z) float32 ...
Attributes:
    Name:                 SV3-1043_PLD2_TAB2
    wms_title:            SV3-1043_PLD2_TAB2
    title:                S-MODE IOP1 2022 Waveglider SV3-1043 real-time data...
    instrument:           Liquid Robotics SV3 Waveglider
    sea_name:             Pacific
    keywords:             Waveglider, S-MODE, Pacific Ocean, Temperature
    time_coverage_start:  2022-09-21T23:47:00
    time_coverage_end:    2022-10-20T13:02:00
    history:              Thu Oct 20 09:13:49 2022: ncrename -v z,z_matrix .....
    NCO:                  netCDF Operators version 4.9.1 (Homepage = http://n...

In [10]:
tmin = np.datetime64('2022-10-10T00:00:00')
tmax = np.datetime64('now')
vmin = -.50 
vmax = .50
levels=np.arange(vmin,vmax,.05)


In [11]:
plt.figure()
plt.set_cmap(cmap=plt.get_cmap('turbo'))
n = 0
ax1 = plt.subplot(len(WG_list),1,len(WG_list))
ax1.set_xlim(tmin,tmax)
zmax=-60
for WG in WG_list:
    n=n+1
    ds = eval('adcp_'+WG)
    ax = plt.subplot(len(WG_list),1,n,sharex=ax1)
    im = plt.pcolor(ds.time.values,ds.depth,ds.current_east,vmin=vmin,vmax=vmax)
    # plt.contourf(ds.time.values,ds.z_matrix[:,1],ds.current_east,levels)
    plt.ylim(zmax, 0)
    plt.text(tmin,zmax+5,WG)
    if n==1: plt.title('East vel')
fig=plt.gcf()
fig.autofmt_xdate()
fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.875, 0.1, 0.025, 0.8])
fig.colorbar(im, cax=cbar_ax)

In [12]:
plt.figure()
plt.set_cmap(cmap=plt.get_cmap('turbo'))
n = 0
ax1 = plt.subplot(len(WG_list),1,len(WG_list))
ax1.set_xlim(tmin,tmax)
for WG in WG_list:
    n=n+1
    ds = eval('adcp_'+WG)
    ax = plt.subplot(len(WG_list),1,n,sharex=ax1)
    im = plt.pcolor(ds.time.values,ds.z_matrix,ds.current_north,vmin=vmin,vmax=vmax)
    # plt.contourf(ds.time.values,ds.z_matrix[:,1],ds.current_east,levels)
    plt.ylim(-60, 0)
    plt.text(tmin,zmax+5,WG)
    if n==1: plt.title('North vel')
fig=plt.gcf()
fig.autofmt_xdate()
fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.875, 0.1, 0.025, 0.8])
fig.colorbar(im, cax=cbar_ax)

OK, that's very cool!  I have all the files cleaned up and have added the lat/lon.  Le't get ready to try finding the ones in the tringle and doing the LS fit.  Maybe a good intermediate step is to plot the vectors on a map.  Or, maybe better would be to do the same plots as the last two above, but showing only the data from the triangle.

In [13]:
# Plot locations
t0 = np.datetime64('2022-10-15T00:00:00')
time_tol = np.timedelta64(5,'D')
plt.figure()
n = 0
zmax=-60
for WG in WG_list:
    n=n+1
    ds = eval('adcp_'+WG)
    tind = np.flatnonzero(np.abs(ds.time-t0)<time_tol)
    plt.plot(ds.Longitude[tind], ds.Latitude[tind])
    # plt.contourf(ds.time.values,ds.z_matrix[:,1],ds.current_east,levels)
    plt.axis('square')
    plt.axis([-124.7, -124.15, 36.8, 37.2])
plt.legend(WG_list)
# Can see two triangles here:
# 36.97, -124.66
# 37.0, -124.63

In [14]:
lon0 = -124.66
lat0 = 36.97
tol = .023


In [15]:
plt.figure()
plt.set_cmap(cmap=plt.get_cmap('turbo'))
n = 0
ax1 = plt.subplot(len(WG_list),1,len(WG_list))
ax1.set_xlim(tmin,tmax)
for WG in WG_list:
    n=n+1
    ds = eval('adcp_'+WG)
    # ds = ds.where(np.logical_and(np.abs(ds.Latitude.values-lat0)<tol, np.abs(ds.Longitude.values-lon0)<tol))
    tind = np.flatnonzero(np.logical_and(np.abs(ds.Latitude.values-lat0)<tol, np.abs(ds.Longitude.values-lon0)<tol))
    ax = plt.subplot(len(WG_list),1,n,sharex=ax1)
    if tind.size == 0: 
        im = plt.axis()
    else:
        im = plt.pcolor(ds.time[tind].values,ds.depth,ds.current_north[:,tind],vmin=vmin,vmax=vmax)
        # plt.contourf(ds.time.values,ds.z_matrix[:,1],ds.current_east,levels)
    plt.ylim(-60, 0)
    plt.text(tmin,zmax+5,WG)
    if n==1: plt.title('North vel')
fig=plt.gcf()
fig.autofmt_xdate()
fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.875, 0.1, 0.025, 0.8])
fig.colorbar(im, cax=cbar_ax)
if savefig:
    plt.savefig(__figdir__+'WG_triangle_north_vel'+'.'+plotfiletype,**savefig_args)

In [16]:
plt.figure()
plt.set_cmap(cmap=plt.get_cmap('turbo'))
n = 0
ax1 = plt.subplot(len(WG_list),1,len(WG_list))
ax1.set_xlim(tmin,tmax)
zmax=-60
for WG in WG_list:
    n=n+1
    ds = eval('adcp_'+WG)
    # ds = ds.where(np.logical_and(np.abs(ds.Latitude.values-lat0)<tol, np.abs(ds.Longitude.values-lon0)<tol))
    tind = np.flatnonzero(np.logical_and(np.abs(ds.Latitude.values-lat0)<tol, np.abs(ds.Longitude.values-lon0)<tol))
    ax = plt.subplot(len(WG_list),1,n,sharex=ax1)
    if tind.size == 0: 
        im = plt.axis()
    else:
        im = plt.pcolor(ds.time[tind].values,ds.depth,ds.current_east[:,tind],vmin=vmin,vmax=vmax)
        # plt.contourf(ds.time.values,ds.z_matrix[:,1],ds.current_east,levels)
    plt.ylim(zmax, 0)
    plt.text(tmin,zmax+5,WG)
    if n==1: plt.title('East vel')
fig=plt.gcf()
fig.autofmt_xdate()
fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.875, 0.1, 0.025, 0.8])
fig.colorbar(im, cax=cbar_ax)
if savefig:
    plt.savefig(__figdir__+'WG_triangle_east_vel'+'.'+plotfiletype,**savefig_args)

In [17]:
np.shape(ds.current_north)

(50, 1862)

OK, WHOI32 has been in the first triangle for about the whole time (since the 12th).  Let's plot a time series of that one.

In [18]:
fig = plt.figure()
ax = plt.subplot(111)
WG = 'WHOI32'
ds = eval('adcp_'+WG)
tind = np.flatnonzero(np.logical_and(np.abs(ds.Latitude.values-lat0)<tol, np.abs(ds.Longitude.values-lon0)<tol))
im = plt.pcolor(ds.time[tind].values,ds.depth,ds.current_east[:,tind],vmin=vmin,vmax=vmax)
plt.ylim(zmax, 0)
plt.text(tmin,zmax+5,WG)
plt.title('East vel for ' + WG + ' when in triangle')
fig.autofmt_xdate()
fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.875, 0.1, 0.025, 0.8])
fig.colorbar(im, cax=cbar_ax)
ax.set_xlim(tmin,tmax)
if 0: #savefig:
    plt.savefig(__figdir__+'WHOI32_triangle_east_vel'+'.'+plotfiletype,**savefig_args)

In [19]:
fig = plt.figure()
ax = plt.subplot(111)
WG = 'WHOI32'
ds = eval('adcp_'+WG)
tind = np.flatnonzero(np.logical_and(np.abs(ds.Latitude.values-lat0)<tol, np.abs(ds.Longitude.values-lon0)<tol))
z0 = -15
zind = np.flatnonzero(np.abs(ds.depth-z0)<1)
plt.plot(ds.time.values[tind],np.squeeze(ds.current_east[zind,tind]))
plt.plot(ds.time.values[tind],np.squeeze(ds.current_north[zind,tind]))
plt.legend(['U','V'])
# plt.contourf(ds.time.values,ds.z_matrix[:,1],ds.current_east,levels)
plt.title(WG+ ' east vel')
plt.ylabel('[m/s]')
fig.autofmt_xdate()
ax.set_xlim(tmin,tmax)
if savefig:
    plt.savefig(__figdir__+'WHOI32_triangle_time_series'+'.'+plotfiletype,**savefig_args)

In [20]:
# %whos

# How to organize code for the fits?

One way to do it would be to dump all of the U, V, lon, lat, time values into vectors and then select the data points that are within some lon/lat/time distance of a given target.  That should work.

In [21]:
len(ds.time)

2694

In [22]:
# dump all of the U, V, lon, lat, time values into vectors and then select the data points that are within some lon/lat/time distance of a given target
# I guess this will have to be Ntimes x Nz, where N = len(time)*len(WG_list)
Ntimes=0
Nz_check = []
z_check = []
for WG in WG_list:
    ds = eval('adcp_'+WG)
    Ntimes=Ntimes+len(ds.time)
    Nz_check.append(len(ds.depth)) # doing this to verify that 
    z_check.append(np.min(ds.depth)) # doing this to verify that 

if np.diff(Nz_check).all()==0 and np.diff(z_check).all()==0 :
    Nz=Nz_check[0]
    z=ds.depth
else:
    raise ValueError('There are different numbers of depth points in the files')

In [23]:
U = np.zeros((Ntimes, Nz))
np.shape(U)

(12724, 50)

In [24]:
U = []
V = []
lon = []
lat = []
time = []
name = []

for WG in WG_list:
    ds = eval('adcp_'+WG)
    #tind = np.flatnonzero(np.logical_and(np.abs(ds.Latitude.values-lat0)<tol, np.abs(ds.Longitude.values-lon0)<tol))
    U.extend(ds.current_east.transpose())
    V.extend(ds.current_north.transpose())
    time.extend(ds.time.values)
    lon.extend(ds.Longitude)
    lat.extend(ds.Latitude)
    name.extend(np.repeat(WG,len(ds.time.values)))

U = np.array(U)
V = np.array(V)
lon = np.array(lon)
lat = np.array(lat)
time = np.array(time)
name = np.array(name)

In [25]:
print(np.shape(lat),np.shape(lon),np.shape(time),np.shape(U),np.shape(V))

(12724,) (12724,) (12724,) (12724, 50) (12724, 50)


In [26]:
z0 = -15
zind = np.flatnonzero(np.abs(z-z0)<1)

lon0 = -124.66
lat0 = 36.96
lon_lat_tol = .023
t0 = np.datetime64('2022-10-13T12:00:00')
time_tol = np.timedelta64(1,'h')



In [27]:
def subset_vel(lon0, lat0, t0, lon_lat_tol, time_tol, zind):
    tind = np.flatnonzero(np.logical_and(np.logical_and(np.abs(lat-lat0)<lon_lat_tol, np.abs(lon-lon0)<lon_lat_tol),np.abs(time-t0)<time_tol))
    lonsub = lon[tind]
    latsub = lat[tind]
    timesub = time[tind]
    Usub = U[tind,zind]
    Vsub = V[tind,zind]
    namesub = name[tind]
    xsub = []
    for n in range(len(lonsub)):
        loni=lonsub[n]
        xsub.append(np.sign(loni-lon0)*gsw.geostrophy.distance((loni,lon0), (lat0,lat0)))
    ysub = []
    for lati in latsub:
        ysub.append(np.sign(lati-lat0)*gsw.geostrophy.distance((lon0,lon0), (lati,lat0)))
    
    return namesub, lonsub, latsub, xsub, ysub, timesub, Usub, Vsub

In [28]:
namesub, lonsub, latsub, xsub, ysub, timesub, Usub, Vsub = subset_vel(lon0,lat0,t0, lon_lat_tol, time_tol, zind)

In [29]:
plt.figure()
# plt.scatter(xsub, ysub,c=Vsub)
plt.quiver(xsub, ysub,Usub, Vsub)
plt.title(', '.join(np.unique(namesub)) + ' data, ' + str(abs(lon0)) + 'W, ' + str(lat0) + 'N, time = ' + str(t0) )
plt.axis('equal')

(-751.2793755339857, 712.2154872333676, 591.420815645389, 1832.323518557612)

In [30]:
plt.figure()
plt.plot(timesub, Usub,'.')

We want to estimate $\frac{\partial u}{\partial x}$, $\frac{\partial u}{\partial y}$, $\frac{\partial v}{\partial x}$, and $\frac{\partial v}{\partial y}$. We will fit a plane to the measured velocity components.  Conceptually, we are thinking of the velocity as being,
\begin{equation}
    u(x,y) = u_0 + \frac{\partial u}{\partial x}\Delta x + \frac{\partial u}{\partial y}\Delta y
\end{equation}

We could write this system of equations in vector form as:
\begin{eqnarray}
  \begin{bmatrix} u_{n=1} \\ u_{n=2} \\ u_{n=3} \end{bmatrix} = u_0 \begin{bmatrix} 1 \\ 1 \\ 1 \end{bmatrix} +\frac{\partial u}{\partial x} \begin{bmatrix} \Delta x_{n=1} \\ \Delta x_{n=2} \\ \Delta x_{n=3} \end{bmatrix} +\frac{\partial u}{\partial y} \begin{bmatrix} \Delta y_{n=1} \\ \Delta y_{n=2} \\ \Delta y_{n=3} \end{bmatrix}
\end{eqnarray}
To simplify the notation, let $\frac{\partial u}{\partial x}\equiv u_x$, $\frac{\partial u}{\partial y}\equiv u_y$, $\Delta x\equiv x$, and $\Delta y \equiv y$.  Then the above equation is the same as
\begin{eqnarray}
  \begin{bmatrix} u_{n=1} \\ u_{n=2} \\ u_{n=3} \end{bmatrix} = \begin{bmatrix} 1 & x_{n=1} & y_{n=1} \\ 1 & x_{n=2} & y_{n=2} \\ 1 & x_{n=3} & y_{n=3} \end{bmatrix} \begin{bmatrix} u_0 \\ u_x \\u_y  \end{bmatrix} \\
  \mathbf{y} \ \ \  = \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \mathbf{E} \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \mathbf{x} \ \ \ \ \
\end{eqnarray}


The ordinary least squares solution is:
\begin{equation}
  \tilde{\mathbf{x}}=\left(\mathbf{E}^T\mathbf{E}\right)^{-1}\mathbf{E}^T\mathbf{y}.
 \label{TW_LS2}
\end{equation}

To think about doing this in python using the numpy.linalg library, it might help to rewrite it as:
\begin{equation}
  \left(\mathbf{E}^T\mathbf{E}\right)\tilde{\mathbf{x}}=\mathbf{E}^T\mathbf{y}.
\end{equation}



In [31]:
# So, let's define E and y
# first row of E is 1 xsub[0] ysub[0]
E=[]
for n in range(len(xsub)):
    E.append([1,xsub[n].item(),ysub[n].item()])

In [32]:
E = np.asarray(E)
y = Usub

In [33]:
np.shape(E)

(24, 3)

In [34]:
A = E.transpose()@E
B = E.transpose()@y

In [35]:
xtilde = np.linalg.solve(A, B) # solves A x = B

In [36]:
u0, ux, uy = xtilde
print('u0 = ' + str(u0) + ', ux = ' + str(ux) + ', uy = ' + str(uy))

u0 = 0.2717793141408435, ux = 2.4442744991038983e-05, uy = -3.646376992010044e-05


In [37]:
def plane_fit_2D(Usub,xsub,ysub):
    ''' 
    Inputs: 
    Usub = scalar field to fit
    xsub = x locations
    ysub = y locations
    
    Outputs:
    Estimates of:
    u0 (value at x=0,y=0)
    ux (x slope of input field)
    uy (y slope of input field)
    '''
    
    E=[]
    for n in range(len(xsub)):
        E.append([1,xsub[n].item(),ysub[n].item()])
    E = np.asarray(E)
    y = Usub
    A = E.transpose()@E
    B = E.transpose()@y
    u0, ux, uy = np.linalg.solve(A, B) # solves A x = B
    
    return u0, ux, uy

In [38]:
u0, ux, uy = plane_fit_2D(Usub,xsub,ysub)

## OK, now we are set up to fit the whole time series!

In [45]:
z0 = -15
zind = np.flatnonzero(np.abs(z-z0)<1)

lon0 = -124.66
lat0 = 36.97
lon_lat_tol = .023
time_tol = np.timedelta64(1,'h')
#t0 = np.datetime64('2022-10-13T12:00:00')

In [40]:
t = np.arange('2022-10-11T12:00:00', '2022-10-16T20:00:00', dtype='datetime64[h]')

In [41]:
adsgfdsg

NameError: name 'adsgfdsg' is not defined

In [46]:
plt.figure()
ax = [-1000, 1000, -1000, 1000]
u0 = []; ux = []; uy = []; v0 = []; vx = []; vy = []
n=0
for t0 in t:
    n=n+1
    plt.clf()
    namesub, lonsub, latsub, xsub, ysub, timesub, Usub, Vsub = subset_vel(lon0,lat0,t0, lon_lat_tol, time_tol, zind)
    # Do Least squares fits
    if len(xsub)>3:
        u0i, uxi, uyi = plane_fit_2D(Usub,xsub,ysub)
        v0i, vxi, vyi = plane_fit_2D(Vsub,xsub,ysub)
    else:
        u0i, uxi, uyi = np.nan,np.nan,np.nan
        v0i, vxi, vyi = np.nan,np.nan,np.nan
    u0.append(u0i);ux.append(uxi);uy.append(uyi)
    v0.append(v0i);vx.append(vxi);vy.append(vyi)
    # Plot
    plt.quiver(xsub, ysub,Usub, Vsub)
    plt.quiver(0,0,u0i,v0i,color='r')    
    plt.axis('square')
    plt.title(str(t0))
    plt.text(-400,850,', '.join(np.unique(namesub)))
    plt.axis(ax)
    if savefig:
        plt.savefig(__figdir__+'triangle_vel_plots_'+str(n)+'.'+plotfiletype,**savefig_args)
    
    



In [ ]:
u0 = np.array(u0)
ux = np.array(ux)
uy = np.array(uy)
v0 = np.array(v0)
vx = np.array(vx)
vy = np.array(vy)

In [ ]:
# !ffmpeg -i triangle_vel_plots_%d.png -r 5 video5.avi

In [ ]:
f = gsw.geostrophy.f(37)
inertial_period = 2*np.pi/f/3600
print('Inertial period is '+ f'{inertial_period:.1f}' + ' hours')

In [ ]:
plt.figure()
plt.plot(t,(uy-vx)/f)
fig=plt.gcf()
fig.autofmt_xdate()
plt.title('$\zeta$/f')

In [ ]:
plt.figure()
plt.plot(t,(ux+vy)/f)
fig=plt.gcf()
fig.autofmt_xdate()
plt.title('$\delta$/f')

Consider the tapered and weighted least-squares solution (Equation 1.125 in the course notes),
\begin{equation}
  \tilde{\mathbf{x}}=\left(\mathbf{E}^T\mathbf{W}^{-1}\mathbf{E}+\mathbf{S}^{-1}\right)^{-1}\left(\mathbf{E}^T\mathbf{W}^{-1}\mathbf{y}+\mathbf{S}^{-1}\mathbf{x_0}\right).
 \label{TW_LS}
\end{equation}
Recall that $\mathbf{W}^{-1}$ is a ''weight matrix'', $\mathbf{S}^{-1}$ is a ''taper matrix'' (which can be thought of as another weight matrix, as we shall see soon), and $\mathbf{x_0}$ is the first guess solution.  Just to simplify the notation and discussion a little bit, we will assume that $\mathbf{x_0}=0$, which would be the case if we know or think that the expectation value $<\mathbf{x}>=0$.  In that case
\begin{equation}
  \tilde{\mathbf{x}}=\left(\mathbf{E}^T\mathbf{W}^{-1}\mathbf{E}+\mathbf{S}^{-1}\right)^{-1}\mathbf{E}^T\mathbf{W}^{-1}\mathbf{y}.
 \label{TW_LS2}
\end{equation}

Again assuming $\mathbf{x_0}=0$, the cost function that led to Equation \ref{TW_LS2} was (Equation 1.193 in the notes):
\begin{equation}
  J=\mathbf{n}^T \mathbf{W}^{-1}\mathbf{n}+\mathbf{x}^T\mathbf{S}^{-1}\mathbf{x}.
 \label{J_TW_LS2}
\end{equation}

Like most complicated equations, we can get a better feel for what the equation means by considering some special cases.  A common special case to consider in matrix problems is one where some matrices are diagonal and square, because these matrices can easily be inverted.  If $\mathbf{W}=a \mathbf{I}$, then $\mathbf{W}^{-1}=\frac{1}{a} \mathbf{I}$. So, let's try letting $\mathbf{W}^{-1}=\frac{1}{\sigma_n^2} \mathbf{I}$ and letting $\mathbf{S}^{-1}=\frac{1}{\Delta_x^2} \mathbf{I}$.  Then, the cost function in Equation \ref{J_TW_LS2} becomes
\begin{equation}
  J=\frac{1}{\sigma_n^2}\mathbf{n}^T \mathbf{n}+\frac{1}{\Delta_x^2}\mathbf{x}^T \mathbf{x},
 \label{J_TW_LS2_simple}
\end{equation}
and Equation \ref{TW_LS2} becomes:
\begin{equation}
  \tilde{\mathbf{x}}=\left(\frac{1}{\sigma_n^2}\mathbf{E}^T \mathbf{E}+\frac{1}{\Delta_x^2}\mathbf{I}\right)^{-1}\frac{1}{\sigma_n^2}\mathbf{E}^T\ \mathbf{y},
  \nonumber
\end{equation}
or,
\begin{equation}
  \tilde{\mathbf{x}}=\left(\mathbf{E}^T\mathbf{E}+\frac{\sigma_n^2}{\Delta_x^2}\mathbf{I}\right)^{-1}\mathbf{E}^T \mathbf{y}.
 \label{TW_LS2_simple}
\end{equation}
If $\sigma_n^2$ is the expected noise variance and $\Delta_x^2$ is the expected solution variance, then we can interpret Equation \ref{J_TW_LS2_simple} as a cost function where we equally penalize (in a normalized sense) the estimated noise variance and the estimated solution variance.  We are simultaneously minimizing the model-data misfit and the solution variance.





The tapering parameter $\sigma_n^2/\Delta_x^2$ can be considered to be an inverse signal-to-noise ratio (SNR), expressing our expectation about the relative variance of the measurement noise and the solution.  In the limit that the tapering parameter is very small (meaning the SNR is high), Equation \ref{TW_LS2_simple} is just the ordinary least squares solution.  If the tapering parameter is small, the tapered least squares solution could also be viewed as a mere computational trick-- by adding a small value to the diagonal of $\mathbf{E}^T\mathbf{E}$, we have guaranteed that the inverse $\left(\mathbf{E}^T\mathbf{E}+\frac{\sigma_n^2}{\Delta_x^2}\mathbf{I}\right)^{-1}$ exists.
